In [126]:
import pandas as pd
from scipy.stats import nbinom
import numpy as np
from joblib import Parallel, delayed
import warnings

# cdf of the truncated negative binomial distribution
def truncNbinomCdf(y, n, p, log=True):
    is_scalar = np.isscalar(y)  # check if y is scalar or array

    if is_scalar:
        y = np.array([y])  # typecast scalar to onedimensional array

    f_zero = nbinom.pmf(0, n, p) # f(0) untruncated density

    # general formula for lower trunc. distributions
    cdf_y = (nbinom.cdf(y, n, p) - nbinom.cdf(0, n, p)) / (1 - f_zero)
    # VERY IMPORTANT STEP: there might be numerical instabilities, which may lead to
    # nbinomo.cdf(x) < nbinom.cdf(0)!!!!! this then leads to negative values of the cdf
    # or nans in the log version (np.log(neg number))
    cdf_y[cdf_y < 0] = 0
    # set values to 0, if y <= 0 (y <=0 not allowed per definition of a 0 truncated count distribution)
    cdf_y[y <= 0] = 0

    # in case of log CDF
    if log: 
        # ignore the 'RuntimeWarning: divide by zero encountered in log' warning (np.log(0))
        warnings.filterwarnings('ignore', category=RuntimeWarning)
        log_cdf_y = np.log(cdf_y) # general formula for lower trunc. distributions
        warnings.filterwarnings('default', category=RuntimeWarning)

        if is_scalar:
            return log_cdf_y[0]  # return scalar, if onedimensional array
        else:
            return log_cdf_y
    # normal CDF
    else:
        if is_scalar:
            return cdf_y[0]  
        else:
            return cdf_y

    

#log.p	logical; if TRUE, probabilities p are given as log(p)    
def qnbinom_trunc(p, nNbinom, pNbinom, log_p=False):

    # n and p have to be greater than zero
    if nNbinom == 0 or pNbinom == 0:
        if not isinstance(p, (list, np.ndarray)):
            return np.nan
        else:
            return np.full(len(p), np.nan)

    # if f(0)=0 no truncation is needed
    if nbinom.pmf(0, nNbinom, pNbinom) == 0:
        return nbinom.ppf(p, nNbinom, pNbinom)
    else:
        # Convert p (quantile) to array if it's a scalar
        if not isinstance(p, (list, np.ndarray)):
            p = np.array([p])
        
        n = len(p) # number of quantiles

        # Set log-probabilities (lower tail)
        if log_p:
            logp = p
        else:
            logp = np.log(p)
        
        # Set output and deal with special cases (outputs NA and Inf)
        quantiles = np.full(n, np.nan)
        nna = ~np.isnan(logp)
        nlogp = logp[nna]
        if len(nlogp) == 0:
            return quantiles
        
        quantiles[nna] = np.full(len(nna), np.inf)
        if np.min(nlogp) >= 0:
            return quantiles

        # calculate mean and variance out of n and p
        mean = (nNbinom * (1 - pNbinom)) / pNbinom
        var = (nNbinom * (1 - pNbinom)) / (pNbinom**2)

        # Set log-CDF vector
        lp_max = np.max(nlogp[nlogp < 0]) # find the highest log quantile to calculate
        p_max = np.exp(lp_max) # highest quantile

        # find an adequate upper limit, starting from the extreme conservative chebychev inequality
        upper = int(mean + np.sqrt(var/(1-np.exp(lp_max)))) #Chebychev inequality

        # if upper < 1000 there is an log(0)=-inf with warning -> ignore this warning
        warnings.filterwarnings('ignore', category=RuntimeWarning)
        # lower the upper limit (saves computation time)
        while truncNbinomCdf(upper-1000, nNbinom, pNbinom, log=False) > p_max:
            upper = upper - 1000

        # after this section warnings are enabled again
        warnings.filterwarnings('default', category=RuntimeWarning)

        yarray = np.arange(1, int(upper)+1) # the y values for which the CDF is going to be calculated
        logcdf = truncNbinomCdf(yarray, nNbinom, pNbinom) # calculate log CDF (faster computation time)

        # Compute output
        for i in range(n): # for all quantiles   
            if nna[i]:     # that are not na
                if logp[i] < 0: # and logp<0 count the number of observations (y): cdf(y) < quantile
                    quantiles[i] = np.sum(logcdf < np.array(logp[i])) + 1 #+1 because 0 is truncated
        
        # Return output
        if len(quantiles) == 1:
            return quantiles[0] # if single quantile is handed over
        else:
            return quantiles

# extrem wichtig
es kommt zu numerischen instabilitäten, wie im unteren Beispiel, wenn n (anzahl erfolge) nicht gerundet wird!!

In [127]:
""" n = 308.3191118596585
p = 0.09206801011370347 """


n = 9252.0
p = 0.9226650803093397

print(nbinom.cdf(2, n, p))
print(nbinom.cdf(0,n,p))
print(nbinom.cdf(2, n, p) - nbinom.cdf(0,n,p))

0.0
5e-324
-5e-324


In [128]:
truncNbinomCdf(2, n, p, True) 

-inf

## Bisektionsverfahren alt

In [129]:
def truncNegBin_PPF(x, n, p, epsilon=1e-6, max_iterations=100):
    # if f(0)=0 no truncation is needed
    if (1 - nbinom.pmf(0, n, p)) == 1:
        return nbinom.ppf(x, n, p)
    else:
        # Define the range of y where the solution might exist
        lower_bound = 0
        upper_bound = 1000000000  # Adjust this based on the expected range of y

        # Bisection method
        for _ in range(max_iterations):
            y = (lower_bound + upper_bound) / 2
            cdf_value = truncNbinomCdf(y, n, p, False)

            if abs(cdf_value - x) < epsilon:
                return np.ceil(y)  # Found a good approximation

            if cdf_value < x:
                lower_bound = y
            else:
                upper_bound = y

        # Return the best approximation if max_iterations is reached
        return np.ceil(y)

def calculate_trunc_nbinom_quantile(quantile, n, p):
    return truncNegBin_PPF(quantile, n, p)

## Vergleich der Funktionen

In [130]:
import time
from scipy.stats import nbinom
from joblib import Parallel, delayed
import numpy as np

# calculation of quantiles
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = np.round(quantiles, 3)

# Example usage
mean = 9.5
var = 3736.5

n = (mean**2) / (var - mean) # equivalent to r
p = mean / var

# Measure execution time for variable 'a'
start_time_a = time.time()
a = nbinom.ppf(quantiles, n, p)
end_time_a = time.time()
execution_time_a = end_time_a - start_time_a

# Measure execution time for variable 'b'
start_time_b = time.time()
b = qnbinom_trunc(quantiles, n, p)  # Assuming this function is defined somewhere in your code
end_time_b = time.time()
execution_time_b = end_time_b - start_time_b

# Measure execution time for variable 'c'
start_time_c = time.time()
trunc_nbinom_quantiles = Parallel(n_jobs=-1)(delayed(calculate_trunc_nbinom_quantile)(quantile, n, p) for quantile in quantiles)
c = np.array(trunc_nbinom_quantiles)
end_time_c = time.time()
execution_time_c = end_time_c - start_time_c

print("Execution time for variable 'a':", execution_time_a, "seconds")
print("Execution time for variable 'b':", execution_time_b, "seconds")
print("Execution time for variable 'c':", execution_time_c, "seconds")

Execution time for variable 'a': 0.0050127506256103516 seconds
Execution time for variable 'b': 0.016991138458251953 seconds
Execution time for variable 'c': 13.86510968208313 seconds


In [131]:
a

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

In [132]:
b

array([1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 

In [133]:
c

array([1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00,
       1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 

In [134]:
b == c

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

## Laufzeit der Funktionen überprüfen

In [135]:
def your_function_to_profile(quantiles, n, p):
    return qnbinom_trunc(quantiles, n, p)

In [136]:
import cProfile
import pstats
cProfile.run('your_function_to_profile(quantiles, n, p)', 'profile_results')

# Die Profilierungsergebnisse lesen und nach der selbst verbrauchten Zeit (tottime) sortieren
stats = pstats.Stats('profile_results')
stats.sort_stats('tottime')  # Sortieren nach der selbst verbrauchten Zeit
stats.print_stats() 

Tue Jul  4 18:19:48 2023    profile_results

         11112 function calls (11098 primitive calls) in 0.028 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1015    0.006    0.000    0.006    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.006    0.006    0.028    0.028 C:\Users\Tobias\AppData\Local\Temp\ipykernel_11696\2950434908.py:46(qnbinom_trunc)
        4    0.005    0.001    0.005    0.001 c:\Users\Tobias\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_discrete_distns.py:334(_cdf)
     1015    0.002    0.000    0.010    0.000 c:\Users\Tobias\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:69(_wrapreduction)
      999    0.002    0.000    0.011    0.000 c:\Users\Tobias\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:2188(sum)
      999    0.001    0.000    0.014    0.000 <__array_function__ internals>:177